### Answering predicate counting queries under ϵ-differential privacy with the matrix mechanism

<img style="float: left;" src="out/ex_workload.PNG">

In [1]:
import numpy as np
import math

First, build workload matrix for the CDF workload.

In [2]:
def cdf_workload(dom):
  # workload matrix representing CDF queries
  # dom: domain size
  return np.tril(np.ones(dom))

W = cdf_workload(4)
print(W)

[[ 1.  0.  0.  0.]
 [ 1.  1.  0.  0.]
 [ 1.  1.  1.  0.]
 [ 1.  1.  1.  1.]]


Next, build a sample data vector.

In [3]:
x = np.array([5,10,2,16])

Then, evaluate queries in CDF workload on sample data.

In [4]:
W @ x   # @ is a matrix multiply

array([  5.,  15.,  17.,  33.])

### Privacy Method #1: Answering a workload with the Laplace Mechanism

<img style="float: left;" src="out/laplace_workload.PNG">

In [5]:
def L1_sensitivity(A):  
	"""Return the L1 sensitivity of strategy matrix A: maximum L1 norm of the columns."""
	return float(np.linalg.norm(A, 1))   

def laplace_mechanism(W, x, epsilon=1.0):
  # laplace mechanism on the input workload W
  true_answer = W @ x
  sens = L1_sensitivity(W)
  noise = np.random.laplace(sens/epsilon)
  return true_answer + noise

laplace_mechanism(W, x)

array([  8.96969435,  18.96969435,  20.96969435,  36.96969435])

### Privacy Method #2: Matrix Mechanism

<img style="float: left;" src="out/mm.PNG">

#### Building strategy matrices
Given workload W, find the query strategy A that minimizes the expected total square error

In [6]:
def identity_strategy(dom):
  # Identity strategy (noisy frequency counts)
	return np.eye(dom, dtype=int)

def buildHierarchical(start, end, n, factors):
	"""Builds a hierarchical strategy matrix with branching
	factor determined by the ordered list 'factoring'
	domain size will be the product of the factors
	(for efficiency, we build list of lists, to be converted to matrix later)"""

	m = [ [0]*n ]
	m[0][start:end+1] = [1]*(end+1 - start) 
	if len(factors) >= 1:
		b = factors.pop(0)
		inc = (end - start + 1) // b
	else:
		return m
	for i in range(start, end+1, inc):
		m = m + buildHierarchical(i, i+inc-1, n, factors[:] )  
	return m

def hier_strategy(dom):
  # dom should be a power of 2
  factors = [2]*(int(math.log(dom))+1)
  return np.array(buildHierarchical(0,dom-1,dom,factors))

In [7]:
identity_strategy(4)

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [8]:
hier_strategy(4)

array([[1, 1, 1, 1],
       [1, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

#### Error Analysis

In [9]:
def total_error(W, A, epsilon=1.0):
  # Total squared error of workload queries in W, using strategy in A
  # Matrix-mechanism error calculation
  Aplus = np.linalg.pinv(A)
  frob_term = np.linalg.norm(W @ Aplus, ord='fro')**2
  sens = L1_sensitivity(A)
  return 2 * (sens/epsilon)**2 * frob_term
  
def laplace_total_error(W, epsilon=1.0):
    # total error of answering the workload using the Laplace mechanism
    return 2.0 * (L1_sensitivity(W)/epsilon)**2 * W.shape[0]

For various domain sizes, compare error on CDF workload using A=Identity and A=Hierarchical

In [10]:
print('dom   Identity', '\t', 'Hier', '\t\t', 'Laplace')  
for dom in [2**i for i in [2,3,4,5,6,7,8]]:
  W_ = cdf_workload(dom)
  A_i = identity_strategy(dom)
  A_h = hier_strategy(dom)
  print(f'{dom:3}', 
        f'{total_error(W_, A_i, 1.0):8.3f}', 
        '\t', 
        f'{total_error(W_, A_h, 1.0):8.3f}',
        '\t', 
        f'{laplace_total_error(W_, 1.0):8.3f}'
       )  

dom   Identity 	 Hier 		 Laplace
  4   20.000 	   46.286 	  128.000
  8   72.000 	  175.543 	 1024.000
 16  272.000 	  303.543 	 8192.000
 32 1056.000 	 1067.711 	 65536.000
 64 4160.000 	 3439.838 	 524288.000
128 16512.000 	 6686.614 	 4194304.000
256 65792.000 	 20267.737 	 33554432.000


These results show that Identity and Hierarchical strategies both perform better than just Laplace across all domains, and Identity is better for smaller ranges while Hierarchical starts to perform better when the domain exceeds 64. Knowing this information, lets use apply the matrix mechanism with the Identity strategy to a small domain example.

#### Matrix Mechanism with the Identity Matrix
<img style="float: left;" src="out/hdmm_workload.PNG">

Given a workload of linear queries (W), the matrix mechanism uses an alternative set of queries, the strategy (A), which are answered privately by a standard mechanism. Answers to the workload queries are then derived from the strategy queries. We use W and A to denote the query workload
and query strategy as well as their matrix representation.

In [11]:
def matrix_mechanism(x, W, A, epsilon=1.0):
    # measure strategy queries in A using laplace mechanism to get y vector
    # y = noisy answers to the queries in A
    y = laplace_mechanism(A, x, epsilon)
    # reconstruct from y to get x_hat
    # x_hat = computed estimate of x that minimizes squared error = pseudo-inverse(A)*y
    Aplus = np.linalg.pinv(A)
    x_hat = y @ Aplus
    # return noisy workload answers: W * x_hat
    return W @ x_hat

x = np.array([5,10,2,16])
W = cdf_workload(4)
A = identity_strategy(4)
matrix_mechanism(x, W, A)

array([  6.05384998,  17.10769995,  20.16154993,  37.21539991])